# Chatbot et RAG

🎯 L'objectif de ce notebook est de créer un chatbot basé sur du RAG "simple".Les documents vectorisés sont en effet de type pdf, sans images ni tableau, et n'intègre aucun sommaire.

## 1- Documents loader

Cette première étape consiste à constituer une base documentaire sur laquelle entrainer notre modèle d'IA.
Dans le cas présent, nous avons utilisé un extrait du rapport climat 2024 sur la construction bas carbone.

In [ ]:
# on installe les packages nécessaires (outils) au projet

!pip install pypdf
!pip install langchain_community

In [21]:
# on utilise un loader (différent selon le type de fichier / le type de contenu à analyser) pour charger les documents

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("../src/Conception et construction bas carbone.pdf")
data = loader.load()
content=data[0]

print(content)

page_content='Conception et construction bas carbone Matériaux bas carbone Le bois, solution de 
stockage carbone dans les constructions Les solutions préfabriquées de Savare, filiale 
spécialisée dans la construction bois, sont déployées dans plusieurs projets en cours, 
comme le nouveau bâtiment tertiaire du CHU de Rennes (Ille-et-Vilaine). In’Cube, le centre 
de recherche et d’innovation de Danone, très ambitieux du fait de ses innovations 
environnementales, a été inauguré à Gif-sur-Yvette (Essonne) en 2023. Eiffage Construction 
a eu no- ___ _La branche Construction a mis en place en 2023 une action intitulée « une 
variante bas carbone par offre déposée » pour les appels d’offres supérieurs à 5 millions 
d’euros. Les variantes bas carbone seront systématisées dans toutes les offres dès cette 
année. tamment recours au bois, utilisé en élément structurel et dans l’assemblage des 
planchers. Compact, équipé de stores extérieurs et de panneaux photovoltaïques, réalisé 
avec 11000 m3

#2- Splitting

Cette seconde étape consiste à découper le texte (nommées "chunk") en petites parties de textes allant pouvoir être vectorisées (traitées) et utilisées par un modèle d'IA (et notamment un LLM).

In [22]:
# il existe différentes stratégies de splitter. nous en utiliserons une appelée "RecursuveCharacterTextSplitter"
# on définit 2 paramètres chunk_size pour la taille des chunks et l'overlap correspondant aux mots en commun entre 2 chunks

from langchain.schema import Document

chunk_size=1000
chunk_overlap=200

from langchain_text_splitters import RecursiveCharacterTextSplitter
rc_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n"," ",""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap)

chunks = rc_splitter.split_text(content.page_content)

docs= [Document(page_content=chunk) for chunk in chunks] # note mme - besoin de convertir les chunks qui sont des string en docs avec page_content et meta données

In [23]:
print(f"Nombre de chunks : {len(docs)}")

Nombre de chunks : 6


#3- Storage et retrieval

Cette troisième étape consiste à vectoriser les chunks précédemment crées et les stocker dans une base de données vectorielle. On utilise pour faire cela des embeddings (réseaux de neurones) et on définit un retriever allant comparer selon une méthode donnée (dans notre cas nous utiliserons la similarité) les informations entrantes (input, la question) et la réponse à donner (output, issu de notre base documentaire).

In [ ]:
!pip install langchain_openai
!pip install langchain_chroma

In [11]:
"""
# depuis collab 
from google.colab import userdata
api_key=userdata.get('OPENAI_API_KEY')
"""

"from google.colab import userdata\napi_key=userdata.get('OPENAI_API_KEY')"

In [12]:
""" 
# depuis visual code
""" 

import os
import openai

api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("Clé API récupérée avec succès !")
else:
    print("Erreur : La clé API n'a pas été trouvée.")

from openai import OpenAI
openai.api_key = api_key


Clé API récupérée avec succès !


### 3.1 - Initialisation de la base de données vectorielles

In [25]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_function = OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-small')

vectorstore = Chroma.from_documents(docs,embedding=embedding_function, persist_directory="./chroma_db") 

# création d'un repertoire persitent pour le streamlit
# Deprecated since version langchain-community==0.1.17: Since Chroma 0.4.x the manual persistence method is no longer supported as 
# # docs are automatically persisted.

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
    )


### 3.2 - Construction d'un prompt template

In [ ]:
!pip install langchain
!pip install openai

In [26]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Tu es un assistant serviable. Consigne pour tes réponses : {guidelines}"),
    HumanMessagePromptTemplate.from_template("{copy}")
])

In [27]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=5, return_messages=True)

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", api_key=api_key)

In [29]:
from langchain.chains import ConversationChain #ConversationChain ne fonctionne pas avec un retriever - passer sur un RetrievalQA
from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    memory=memory,
    return_source_documents=False, # je n'arrive pas à faire fonctionner les souces_documents
    output_key="result"  # nécessaire pour indiquer que l'on souhaite avoir uniquement ce résultat en mémoire -- beug sinon
)

### 3.3 - Utilisation de la chaine

In [ ]:
# essai sans memoire avec chaine simple - à ne pas relancer
response = rag_chain.invoke("Qu'est ce que CARL ? ") # OK
print(response.content)

D'après les documents fournis, CARL est une interface numérique utilisée dans les métiers de la route depuis 2021. Cependant, les documents ne fournissent pas plus de détails sur les fonctionnalités ou l'usage précis de cette interface.


In [ ]:
# essai sans memoire avec chaine simple - à ne pas relancer
response = rag_chain.invoke("Qu'est ce que  Nooco ? ") # NOK
print(response.content)

Je suis désolé, mais les documents fournis ne donnent pas d'informations précises sur ce qu'est Nooco. Il serait peut-être utile de rechercher des informations supplémentaires pour obtenir une réponse précise à cette question.


In [ ]:
# essai sans memoire avec chaine simple - à ne pas relancer
response = rag_chain.invoke("Qui travaille avec Logetex Carbone ? ") # OK
print(response.content)

La branche Construction travaille avec la solution Logetex Carbone.


In [30]:
# essai avec memoire avec RetrievalQA - à lancer
response = rag_chain({"query": "Quel type d'isolant à utiliser Eiffage sur le Village des Athlètes ??"})
result = response["result"]

print("Résultat:", result)


Résultat: Eiffage Construction a utilisé des isolants biosourcés à grande échelle sur le chantier du Village des athlètes. Plus précisément, des façades à ossature bois ont été isolées avec de la laine de bois.


In [20]:
# essai avec memoire avec RetrievalQA - à lancer -- vérification de la mémoire
response_2 = rag_chain({"query":"Peux-tu rappeler ce que nous venons de dire ?"})
print(response_2)

{'query': 'Peux-tu rappeler ce que nous venons de dire ?', 'history': [HumanMessage(content="Quel type d'isolant à utiliser Eiffage sur le Village des Athlètes ??", additional_kwargs={}, response_metadata={}), AIMessage(content='Eiffage a utilisé des isolants biosourcés pour le Village des athlètes, plus précisément de la laine de bois.', additional_kwargs={}, response_metadata={})], 'result': "Le rapport climat Eiffage 2024 mentionne plusieurs initiatives pour réduire les émissions de carbone. Nové, une coentreprise entre Eiffage et Arcade-VYV, a commencé la rénovation du parc de logements du ministère des Armées en janvier 2023. Eiffage va expérimenter la méthodologie Réemploi en rénovation du label bas carbone, qui favorise l'utilisation de matériaux biosourcés et le réemploi dans les projets de rénovation, et qui génère des crédits carbone. Cette expérimentation est financée par Icade, en collaboration avec la Coopérative Carbone, le ministère de la Transition écologique et le Cent

# 4 - Création de l'interface

Voir app.py pour l interface streamlit